In [3]:
# Import dependencies
import pandas as pd
import csv
import re

In [4]:
# open profiles.csv and read into a DataFrame
df = pd.read_csv('data\\profiles.csv')
# display the first few rows
df.head()

,bioguide_id,first_name,last_name,birth_date,death_date,profile
0,A000001,Fred,Aandahl,1897-04-09,1966-04-07,A Representative from North Dakota; born in Li...
1,A000002,Watkins,Abbitt,1908-05-21,1998-07-13,A Representative from Virginia; born in Lynchb...
2,A000003,Joel,Abbot,1776-03-17,1826-11-19,a Representative from Georgia; born in Ridgefi...
3,A000004,Amos,Abbott,1786-09-10,1868-11-02,A Representative from Massachusetts; born in A...
4,A000005,Joseph,Abbott,1840-01-15,1908-02-11,A Representative from Texas; born near Decatur...


Some dates are incomplete and have only the year. Let's find them and update if we can

In [5]:
# find birth_date entries that are not in YYYY-MM-DD format
# an AI assistant was used to help me write this regex
invalid_birth_dates = df[~df['birth_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
print("Invalid birth_date entries:")
print(invalid_birth_dates)
# create a list of bioguide_ids with invalid birth_date entries
invalid_ids = invalid_birth_dates['bioguide_id'].tolist()

Invalid birth_date entries:
      bioguide_id first_name last_name birth_date death_date  \
5         A000006     Joseph    Abbott       1825       1881   
6         A000007     Josiah    Abbott       1814       1891   
8         A000009      James    Abdnor       1923       2012   
9         A000010      Hazel      Abel       1888       1966   
10        A000011      Homer     Abele       1916       2000   
...           ...        ...       ...        ...        ...   
13025     Z000010      Roger      Zion       1921       2019   
13028     Z000013     Edward  Zorinsky       1928       1987   
13029     Z000014      Edwin    Zschau       1940        NaN   
13030     Z000015       John     Zubly       1724       1781   
13031     Z000016       John     Zwach       1907       1990   

                                                 profile  
5      A Senator from North Carolina; born in Concord...  
6      A Representative from Massachusetts; born in C...  
8      A Representative an

Found 4402 entries with incomplete birthdates. The data is often found in the profileText field. The profileText was added into the dataset.

In [7]:
# need to extract the birthdates from profileText
# Options: 1. LLM to extract dates 2. regex patterns to find dates 3. combination of both
# a invalid_ids list had been created above to help with this process
# First, try regex patterns to extract dates, claude sonnet 4.5 used to help write these patterns

# find the oldest date in the profileText for each invalid entry
date_patterns = [
    r'(\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b)',  # MM/DD/YYYY or DD/MM/YYYY
    r'(\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b)',    # YYYY/MM/DD
    r'(\b\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4}\b)',  # DD Month YYYY
    r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{1,2},\s\d{4}\b)'  # Month DD, YYYY
]
extracted_dates = {}
for bioguide_id in invalid_ids:
    profile_text = df.loc[df['bioguide_id'] == bioguide_id, 'profile'].values[0]
    found_dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, profile_text)
        found_dates.extend(matches)
    # Convert found dates to datetime objects and find the earliest
    date_objects = []
    for date_str in found_dates:
        for fmt in ("%m/%d/%Y", "%d/%m/%Y", "%Y/%m/%d", "%d %B %Y", "%B %d, %Y"):
            try:
                date_obj = pd.to_datetime(date_str, format=fmt, errors='coerce')
                if pd.notnull(date_obj):
                    date_objects.append(date_obj)
                    break
            except:
                continue
    if date_objects:
        earliest_date = min(date_objects)
        extracted_dates[bioguide_id] = earliest_date.strftime('%Y-%m-%d')
        # Update the DataFrame with the extracted date
        df.loc[df['bioguide_id'] == bioguide_id, 'birth_date'] = earliest_date.strftime('%Y-%m-%d')
    else:
        extracted_dates[bioguide_id] = None
        print(f"No valid date found for {bioguide_id} in profileText.")
print("Extracted birth dates:")
print(extracted_dates)
# Save to CSV
df.to_csv('data\\profiles_cleaned.csv', index=False)




No valid date found for A000241 in profileText.
No valid date found for B001047 in profileText.
No valid date found for G000058 in profileText.
No valid date found for H000280 in profileText.
No valid date found for H000414 in profileText.
No valid date found for L000536 in profileText.
No valid date found for M000564 in profileText.
No valid date found for M001017 in profileText.
No valid date found for P000020 in profileText.
No valid date found for S000605 in profileText.
No valid date found for W000701 in profileText.
Extracted birth dates:
{'A000006': '1825-07-15', 'A000007': '1814-11-01', 'A000009': '1923-02-13', 'A000010': '1888-07-10', 'A000011': '1916-11-21', 'A000012': '1851-03-04', 'A000018': '1920-07-24', 'A000020': '1827-01-11', 'A000023': '1850-05-20', 'A000028': '1875-10-29', 'A000031': '1927-01-13', 'A000033': '1824-04-10', 'A000040': '1848-09-16', 'A000043': '1830-01-06', 'A000046': '1899-01-08', 'A000049': '1774-05-27', 'A000062': '1892-08-20', 'A000066': '1838-10-30'

In [9]:
# find birth_date entries that are not in YYYY-MM-DD format
# an AI assistant was used to help me write this regex
invalid_birth_dates = df[~df['birth_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
print("Invalid birth_date entries:")
print(invalid_birth_dates)
# create a list of bioguide_ids with invalid birth_date entries
invalid_ids = invalid_birth_dates['bioguide_id'].tolist()
# counnt of invalid entries
print(f"Count of invalid birth_date entries: {len(invalid_ids)}")

Invalid birth_date entries:
      bioguide_id first_name last_name birth_date death_date  \
231       A000241   Benjamin    Andrew       1730       1799   
1416      B001047       John      Bull       1740       1802   
4308      G000058     Joseph   Gardner       1752       1794   
5135      H000280    William  Harrison        NaN        NaN   
5269      H000414   Jonathan    Hazard       1744       1824   
7269      L000536     Thomas     Lynch       1727       1776   
7903      M000564  Alexander  McMillan        NaN       1817   
8355      M001017       John    Morton       1724       1777   
8973      P000020       Mann      Page       1749       1781   
10823     S000605     Robert     Smith       1800       1876   
12819     W000701     Joseph      Wood       1712       1791   

                                                 profile  
231    A Delegate from Georgia; born in Dorchester, S...  
1416   A Delegate from South Carolina; born in Prince...  
4308   A Delegate from Pen

11 birthdates were not found. This is acceptable, analysis is for the 1960's to present.

In [10]:
# confrim that there are no invalid entries left
invalid_birth_dates = df[~df['birth_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
print("Invalid birth_date entries after cleaning:")
print(invalid_birth_dates)

Invalid birth_date entries after cleaning:
      bioguide_id first_name last_name birth_date death_date  \
231       A000241   Benjamin    Andrew       1730       1799   
1416      B001047       John      Bull       1740       1802   
4308      G000058     Joseph   Gardner       1752       1794   
5135      H000280    William  Harrison        NaN        NaN   
5269      H000414   Jonathan    Hazard       1744       1824   
7269      L000536     Thomas     Lynch       1727       1776   
7903      M000564  Alexander  McMillan        NaN       1817   
8355      M001017       John    Morton       1724       1777   
8973      P000020       Mann      Page       1749       1781   
10823     S000605     Robert     Smith       1800       1876   
12819     W000701     Joseph      Wood       1712       1791   

                                                 profile  
231    A Delegate from Georgia; born in Dorchester, S...  
1416   A Delegate from South Carolina; born in Prince...  
4308   A De

In [11]:
# verify that the DataFrame has been cleaned
df.head(50)

,bioguide_id,first_name,last_name,birth_date,death_date,profile
0,A000001,Fred,Aandahl,1897-04-09,1966-04-07,A Representative from North Dakota; born in Li...
1,A000002,Watkins,Abbitt,1908-05-21,1998-07-13,A Representative from Virginia; born in Lynchb...
2,A000003,Joel,Abbot,1776-03-17,1826-11-19,a Representative from Georgia; born in Ridgefi...
3,A000004,Amos,Abbott,1786-09-10,1868-11-02,A Representative from Massachusetts; born in A...
4,A000005,Joseph,Abbott,1840-01-15,1908-02-11,A Representative from Texas; born near Decatur...
5,A000006,Joseph,Abbott,1825-07-15,1881,A Senator from North Carolina; born in Concord...
6,A000007,Josiah,Abbott,1814-11-01,1891,A Representative from Massachusetts; born in C...
7,A000008,Nehemiah,Abbott,1804-03-29,1877-07-26,"A Representative from Maine; born in Sidney, M..."
8,A000009,James,Abdnor,1923-02-13,2012,A Representative and a Senator from South Dako...
9,A000010,Hazel,Abel,1888-07-10,1966,"A Senator from Nebraska; born in Plattsmouth, ..."


In [13]:
# save the cleaned DataFrame to a new CSV file
df.to_csv('data\\profiles_cleaned.csv', index=False)